## train.csv 파일의 input과 output을 바꾼 후 파인 튜닝을 통해 리뷰를 난독화하는 모델을 가지고 step 1에서 생성된 리뷰들을 난독화

In [1]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [2]:
BASE_MODEL = "beomi/gemma-ko-7b"
FINETUNE_MODEL = "./gemma-ko-7b-finetuning-encoder"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
test = pd.read_csv('./data/generated_reviews.csv', encoding="utf-8-sig")

In [4]:
pipe_finetuned = pipeline(task="text-generation", model=finetune_model, tokenizer=tokenizer)

Device set to use cuda:0


In [5]:
restored_reviews = []

# 모든 프롬프트를 리스트로 저장
prompts = [
    (r"""<bos><start_of_turn>user
글자 수와 띄어쓰기를 유지하여 리뷰를 난독화하세요:

{}<end_of_turn>
<start_of_turn>model
""".format(row['output']))
    for _, row in test.iterrows()
]

# 한 번에 병렬 처리
generated_results = pipe_finetuned(
    prompts,  # 여러 개의 프롬프트를 리스트로 입력
    num_return_sequences=1,
    temperature=1.0,
    top_p=0.9,
    max_new_tokens=256,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    batch_size=4  # 병렬 처리 크기 조절 가능
)

# 생성된 결과 처리
for index, (gen, prompt) in enumerate(zip(generated_results, prompts)):
    generated_text = gen[0]['generated_text']
    
    # 프롬프트 부분 제외하고 결과 추출
    result = generated_text[len(prompt):].strip()
    result = result.split('<end_of_turn>')[0]
    
    # 글자 수 검증 후 저장
    original_query = test.iloc[index]['output']
    if len(result) == len(original_query):
        restored_reviews.append(result)
    else:
        print(f"index {index} error occurred")
        print(result)
        print(original_query)
        restored_reviews.append('error')

KeyboardInterrupt: 

In [ ]:
retest_indices = [i for i in range(len(restored_reviews)) if restored_reviews[i] == 'error']

saved_reviews = []
unsaved_indices = []

prompts = [
    (r"""<bos><start_of_turn>user
글자 수와 띄어쓰기를 유지하여 리뷰를 난독화하세요:

{}<end_of_turn>
<start_of_turn>model
""".format(test['output'][i]))
    for i in retest_indices
]

# 텍스트 생성
generated_results = pipe_finetuned(
    prompts,
    num_return_sequences=1,
    temperature=0.9,
    top_p=0.9,
    max_new_tokens=1024,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    batch_size=2
)

for index, (gen, prompt) in enumerate(zip(generated_results, prompts)):
    i = retest_indices[index]
    
    # 생성된 텍스트에서 출력 부분 추출
    generated_text = gen[0]['generated_text']
    result = generated_text[len(prompt):].strip()
    result = result.split('<end_of_turn>')[0]
    if len(result) == len(test['output'][i]):
        saved_reviews.append((i, result))
    else:
        print(f"index {i} error occured")
        print(result)
        print(test['output'][i])
        unsaved_indices.append(i)

In [ ]:
for i, review in saved_reviews:
    restored_reviews[i] = review
    
test['input'] = restored_reviews

for i in range(len(test)):
    for c1, c2 in zip(test['output'][i], test['input'][i]):
        if (c1 == ' ') != (c2 == ' '):
            if i not in unsaved_indices:
                print(f"index {i} error occured")
                unsaved_indices.append(i)
            break

In [ ]:
df = test.drop(unsaved_indices)

df.to_csv('./data/generated_dataset3.csv', index = False, encoding = 'utf-8-sig')